In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import nltk
from nltk.stem import WordNetLemmatizer 

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
'''from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train', shuffle = True)
newsgroups_test = fetch_20newsgroups(subset='test', shuffle = True)'''

"from sklearn.datasets import fetch_20newsgroups\nnewsgroups_train = fetch_20newsgroups(subset='train', shuffle = True)\nnewsgroups_test = fetch_20newsgroups(subset='test', shuffle = True)"

In [3]:
'''print(list(newsgroups_train.target_names))'''


'print(list(newsgroups_train.target_names))'

In [4]:
'''(newsgroups_train.data[:2])'''


'(newsgroups_train.data[:2])'

In [5]:
df = pd.read_csv('BlogDataset.csv')

In [7]:
df = df[['Title','Content']]

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271 entries, 0 to 270
Data columns (total 2 columns):
Title      271 non-null object
Content    271 non-null object
dtypes: object(2)
memory usage: 4.3+ KB


In [11]:
def preprocess(text):
    return [w for w in gensim.utils.simple_preprocess(text) if w not in gensim.parsing.preprocessing.STOPWORDS and len(w)>3]
def lemmatize(text):
    return [WordNetLemmatizer().lemmatize(w) for w in text]

In [12]:
preproc_doc = []
for s in df['Content']:
    preproc_doc.append(lemmatize(preprocess(s)))
    

In [14]:
preproc_doc[:2]

[['kudos',
  'mike',
  'upchurch',
  'latest',
  'youtube',
  'video',
  'freeway',
  'hell',
  'minute',
  'voiceovers',
  'mike',
  'judge',
  'starring',
  'chris',
  'fairbanks',
  'dore',
  'lynch',
  'emily',
  'maya',
  'mill',
  'electric',
  'television',
  'present',
  'lost',
  'motorbike',
  'gang',
  'film',
  'freeway',
  'hell',
  'shocking',
  'wrenchingly',
  'violent',
  'morality',
  'tale',
  'technological',
  'worker',
  'quits',
  'join',
  'gang',
  'take',
  'wild',
  'ride',
  'seedy',
  'underworld',
  'rowdy',
  'wild',
  'woman',
  'motorbike',
  'image',
  'youtubecitizen',
  'brainchild',
  'brian',
  'innovator',
  'residence',
  'program',
  'library',
  'congress',
  'goal',
  'project',
  'simple',
  'provide',
  'free',
  'audio',
  'video',
  'sample',
  'encourage',
  'creativity',
  'remixing',
  'word',
  'cultivate',
  'creation',
  'transformative',
  'music',
  'free',
  'audio',
  'video',
  'material',
  'justify',
  'love',
  'trip',
  'tun

In [15]:
dwords = gensim.corpora.Dictionary(preproc_doc)

In [16]:
c = 0
for k,v in dwords.iteritems():
    c+=1
    print(k,v)
    if c==10:
        break

0 action
1 apple
2 assembling
3 assortment
4 audio
5 awesome
6 backing
7 batman
8 beeble
9 belt


In [18]:
bow = [dwords.doc2bow(s) for s in preproc_doc]

In [19]:
bow

[[(0, 1),
  (1, 3),
  (2, 1),
  (3, 1),
  (4, 2),
  (5, 1),
  (6, 2),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 2),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 2),
  (42, 2),
  (43, 1),
  (44, 2),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 2),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 2),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 3),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 2),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 2),
  (73, 1),
  (74, 1),
  (75, 2),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 2),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 2),
  (89, 1),
  (90, 1),
  (91, 1)

In [20]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow)
corpus_tfidf = tfidf[bow]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.06467823399113073),
 (1, 0.000728963382890285),
 (2, 0.0005712372598516039),
 (3, 0.0005712372598516039),
 (4, 0.15378725417007874),
 (5, 0.0004887109683061957),
 (6, 0.0009774219366123914),
 (7, 0.0004887109683061957),
 (8, 0.1222840474324256),
 (9, 0.0005712372598516039),
 (10, 0.00032458855517091125),
 (11, 0.10715390731663019),
 (12, 0.06994236232143039),
 (13, 0.07980837410692616),
 (14, 0.0009774219366123914),
 (15, 0.0005712372598516039),
 (16, 0.1222840474324256),
 (17, 0.05291292237084681),
 (18, 0.10715390731663019),
 (19, 0.07689362708503937),
 (20, 0.0004887109683061957),
 (21, 0.0004887109683061957),
 (22, 0.0004064955109869275),
 (23, 0.06317224530378535),
 (24, 0.0871529500179779),
 (25, 0.1222840474324256),
 (26, 0.00016169094757485385),
 (27, 0.0004064955109869275),
 (28, 0.0004887109683061957),
 (29, 0.06467823399113073),
 (30, 0.0004887109683061957),
 (31, 0.042882168836486365),
 (32, 0.06176348696924397),
 (33, 0.09830334271823303),
 (34, 0.000571237259851603

In [22]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=dwords, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.001*"prince" + 0.001*"performance" + 0.001*"arbery" + 0.001*"musician" + 0.001*"killer" + 0.001*"theme" + 0.001*"game" + 0.001*"division" + 0.001*"available" + 0.001*"youtube"
Topic: 1 Word: 0.001*"video" + 0.001*"excel" + 0.001*"conspiracy" + 0.001*"isolate" + 0.001*"wuhan" + 0.001*"slogan" + 0.001*"genome" + 0.001*"complete" + 0.001*"coronavirus" + 0.000*"dancing"
Topic: 2 Word: 0.001*"robot" + 0.001*"knife" + 0.001*"cracknell" + 0.000*"facebook" + 0.000*"drug" + 0.000*"comic" + 0.000*"seal" + 0.000*"giphy" + 0.000*"powerpoint" + 0.000*"trump"
Topic: 3 Word: 0.001*"speech" + 0.001*"mike" + 0.001*"freeway" + 0.001*"hell" + 0.001*"notorious" + 0.001*"biggie" + 0.001*"subreddit" + 0.001*"model" + 0.001*"generated" + 0.001*"grave"
Topic: 4 Word: 0.002*"trump" + 0.001*"president" + 0.001*"coronavirus" + 0.001*"donald" + 0.001*"republican" + 0.001*"state" + 0.001*"medium" + 0.001*"survey" + 0.001*"said" + 0.001*"veteran"


In [24]:
bow_test = dwords.doc2bow(lemmatize(preprocess(df['Content'][0])))

for index, score in sorted(lda_model_tfidf[bow_test], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

Score: 0.5065594911575317	 Topic: 0.001*"speech" + 0.001*"mike" + 0.001*"freeway" + 0.001*"hell" + 0.001*"notorious" + 0.001*"biggie" + 0.001*"subreddit" + 0.001*"model" + 0.001*"generated" + 0.001*"grave"
Score: 0.43147462606430054	 Topic: 0.001*"robot" + 0.001*"knife" + 0.001*"cracknell" + 0.000*"facebook" + 0.000*"drug" + 0.000*"comic" + 0.000*"seal" + 0.000*"giphy" + 0.000*"powerpoint" + 0.000*"trump"
Score: 0.059637267142534256	 Topic: 0.001*"prince" + 0.001*"performance" + 0.001*"arbery" + 0.001*"musician" + 0.001*"killer" + 0.001*"theme" + 0.001*"game" + 0.001*"division" + 0.001*"available" + 0.001*"youtube"


In [25]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model_tfidf, corpus=bow, texts=preproc_doc)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,3.0,0.5066,"speech, mike, freeway, hell, notorious, biggie...","[kudos, mike, upchurch, latest, youtube, video..."
1,1,1.0,0.6536,"video, excel, conspiracy, isolate, wuhan, slog...","[ethos, forever, entirely, possible, heard, te..."
2,2,3.0,0.8443,"speech, mike, freeway, hell, notorious, biggie...","[flow, biggie, grave, join, subreddit, associa..."
3,3,3.0,0.9610,"speech, mike, freeway, hell, notorious, biggie...","[andrew, cotter, lockdown, fame, narrating, mu..."
4,4,4.0,0.7798,"trump, president, coronavirus, donald, republi...","[united, state, department, veteran, affair, s..."
5,5,3.0,0.4735,"speech, mike, freeway, hell, notorious, biggie...","[wall, street, journal, reported, friday, fede..."
6,6,0.0,0.9956,"prince, performance, arbery, musician, killer,...","[nook, special, announcement, twitter, friday,..."
7,7,4.0,0.7698,"trump, president, coronavirus, donald, republi...","[want, interesting, little, science, experimen..."
8,8,4.0,0.6891,"trump, president, coronavirus, donald, republi...","[word, today, impeached, manifestly, unfit, pr..."
9,9,4.0,0.9379,"trump, president, coronavirus, donald, republi...","[miss, piggy, kermit, frog, favorite, muppets,..."


In [26]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tfidf, bow, dictionary=lda_model_tfidf.id2word)
vis

C:\Users\jinit\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.017770 -0.001251       1        1  46.319347
0      0.006477  0.003390       2        1  16.685068
3      0.005956 -0.012825       3        1  14.432743
2      0.002140  0.001873       4        1  11.451632
1      0.003198  0.008812       5        1  11.111214, topic_info=     Category       Freq            Term      Total  loglift  logprob
term                                                                 
343   Default  20.000000          speech  20.000000  30.0000  30.0000
42    Default  17.000000         freeway  17.000000  29.0000  29.0000
75    Default  17.000000            mike  17.000000  28.0000  28.0000
4628  Default   9.000000           excel   9.000000  27.0000  27.0000
50    Default  17.000000            hell  17.000000  26.0000  26.0000
340   Default  12.000000       notorious  12.000000  25.0000  25.0000
333   Default  12.000000          biggie  12.000000  24.0000  24.0000
344   Default  12.000000       subreddit  12.000000  23.0000  23.0000
336   Default  12.000000       generated  12.000000  22.0000  22.0000
640   Default  17.000000          prince  17.000000  21.0000  21.0000
337   Default  13.000000           grave  13.000000  20.0000  20.0000
140   Default  30.000000           video  30.000000  19.0000  19.0000
341   Default  12.000000         pattern  12.000000  18.0000  18.0000
342   Default  13.000000          reddit  13.000000  17.0000  17.0000
352   Default  13.000000             dog  13.000000  16.0000  16.0000
335   Default  13.000000            flow  13.000000  15.0000  15.0000
339   Default  15.000000           model  15.000000  14.0000  14.0000
2477  Default  11.000000      conspiracy  11.000000  13.0000  13.0000
346   Default  12.000000         trained  12.000000  12.0000  12.0000
332   Default  13.000000      associated  13.000000  11.0000  11.0000
5239  Default  12.000000          arbery  12.000000  10.0000  10.0000
2491  Default   8.000000         isolate   8.000000   9.0000   9.0000
2511  Default   8.000000           wuhan   8.000000   8.0000   8.0000
2108  Default  12.000000           robot  12.000000   7.0000   7.0000
347   Default  16.000000           voice  16.000000   6.0000   6.0000
2487  Default   8.000000          genome   8.000000   5.0000   5.0000
1823  Default   8.000000          slogan   8.000000   4.0000   4.0000
363   Default  11.000000  vocalsynthesis  11.000000   3.0000   3.0000
345   Default  13.000000            text  13.000000   2.0000   2.0000
334   Default  15.000000         channel  15.000000   1.0000   1.0000
...       ...        ...             ...        ...      ...      ...
2970   Topic5   2.245333      toothbrush   6.813299   1.0872  -7.9883
5802   Topic5   2.229708        jalopnik   6.782528   1.0847  -7.9953
2505   Topic5   2.229420         trouble   6.788306   1.0838  -7.9954
7264   Topic5   2.208097        beaumont   6.760188   1.0783  -8.0050
5894   Topic5   2.186253         florian   6.738647   1.0715  -8.0149
180    Topic5   2.555492            cash   7.918570   1.0662  -7.8589
2488   Topic5   2.122124          hazard   6.683830   1.0499  -8.0447
2484   Topic5   2.119823         favored   6.683770   1.0489  -8.0458
2503   Topic5   2.119650        syndrome   6.683862   1.0488  -8.0459
2483   Topic5   2.118235       excluding   6.683879   1.0481  -8.0466
2501   Topic5   2.117074            spat   6.683962   1.0475  -8.0471
2497   Topic5   2.299350         quarter   7.450269   1.0216  -7.9645
2489   Topic5   2.826468            hour  10.552585   0.8799  -7.7581
2508   Topic5   2.329431         warning   7.822003   0.9859  -7.9515
140    Topic5   4.816671           video  30.478331   0.3523  -7.2251
1511   Topic5   2.328292           mayor   7.949350   0.9693  -7.9520
610    Topic5   3.203494            game  21.831890   0.2781  -7.6329
194    Topic5   3.112769           covid  21.756210   0.2528  -7.6616
182    To

In [ ]:
def pipeline(doc):
    bow_doc = dwords.doc2bow(lemmatize(preprocess(doc)))
    for index, score in sorted(lda_model_tfidf[bow_test], key=lambda tup: -1*tup[1]):
        print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))
    
    df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model_tfidf, corpus=bow, texts=preproc_doc)
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']    
    return [i for i in df_dominant_topic['Keywords'] if i in df_dominant_topic['Text']]